## Adding Programmabilty to SQL with Templates using Python and `jinja2`    

The following is a brief tutorial and demo of how to programmatically generate SQL.

```
"Languages shape the way we think and determines what we can think about." 
~ Benjamin Wharf, American linguist
```
```
Keep it D.R.Y. Don't Repeat Yourself.
```

**Why use templating over SQL built-in functionality like SQL variables, stored procedures, and loops?**
While you can use many programming features in SQL -- variables, loops, conditionals -- it doesn't mean that you always should. In SQL, tables and rows are the main abstractions. If you want to loop through a list of elements or handle more complicated data structures, SQL might get in the way. Additionally, say, you have several tables following a convention of table_<year> (table_2020, table_2021, etc). In SQL, to parameterize the year, you likely have to construct strings dynamically. This sort of programming and string manipulation becomes much more tedious than necessary compared to a procedural programming language (Python, Python+templating language).

**Why not to use templating?**
If all you need is a handful or less of parameterization (variables) or loops, SQL stored procedures and SQL loops might be all that you want without the added glue scripting from Python. In analytics, SQL is often the primary language so depending on your collaborators' background, the jump from basic SQL to intermediate/advanced SQL might be smaller than the jump to Python or another procedural programming langugage.
<br>Assuming a little Python familiarity, I'd argue that even for basic programmability, SQL with templates is easier to maintain and read.

### Contents
* Variables
* Looping
* Conditionals
* Special loop variables
* Note on SQL injection for web applications

### Resources
* https://jinja.palletsprojects.com/en/3.0.x/templates
* https://realpython.com/primer-on-jinja-templating

### Variables
`{{...}}` double curly braces

In [1]:
# standard Python string.replace()
sql_code = "select * from orders where customer_id = '@customer_id'"
sql_code = sql_code.replace('@customer_id', '123')
print(sql_code)

select * from orders where customer_id = '123'


In [2]:
from jinja2 import Template

sql_template = "select * from orders where customer_id = {{customer_id}}"
sql_code = Template(sql_template).render(customer_id=123)
print(sql_code)

select * from orders where customer_id = 123


In [3]:
from jinja2 import Template

sql_template = """
select * 
from data
where client_id = {{client_id}}
and year_month = '{{year_month}}'
"""

params = {'client_id': 123,
          'year_month': '2022-01-01'}

# if we have many variables, we can use dictionary unpacking:
# sames as: 
    # sql_code = Template(sql_template).render(client_id=params['client_id'], year_month=params['year_month'], ...)
sql_code = Template(sql_template).render(**params)
print(sql_code)


select * 
from data
where client_id = 123
and year_month = '2022-01-01'


### Looping
* `{...}` single curly braces with % symbol, for/endfor keywords
* Ex: `{% for thing in things %} here is {{ thing }} {% endfor %}`
* Use hyphens inside of % symbol to control whitespace. Experiment with adding or removing the hyphen.

In [4]:
from jinja2 import Template

sql_template = """
select
{% for year in years %}
year_{{ year }}
{%- endfor %}

from a_table;
"""
years = [2010, 2011, 2012, 2013, 2014, 2015]

sql_code = Template(sql_template).render(years=years)
print(sql_code)


select

year_2010
year_2011
year_2012
year_2013
year_2014
year_2015

from a_table;


In [5]:
# Compare to how you might implement dynamic table names in SQL
    # we will address how to account for removing the last "UNION ALL" in the next session

from jinja2 import Template

sql_template = """
{%- for year in years %}
select * from data_{{year}} union all
{%- endfor %}
"""
years = range(2010, 2026)
sql_code = Template(sql_template).render(years=years)
print(sql_code)


select * from data_2010 union all
select * from data_2011 union all
select * from data_2012 union all
select * from data_2013 union all
select * from data_2014 union all
select * from data_2015 union all
select * from data_2016 union all
select * from data_2017 union all
select * from data_2018 union all
select * from data_2019 union all
select * from data_2020 union all
select * from data_2021 union all
select * from data_2022 union all
select * from data_2023 union all
select * from data_2024 union all
select * from data_2025 union all


### Conditionals
* `{...}` single curly braces with % symbol, if-else-endif keywords. 
* Ex: `{% if thing==0 %} 'zero' {% else %} 'non-zero' {% endif %}`

In [6]:
from jinja2 import Template

sql_template = """
select * 
from claims_data
where client_id = {{client_id}}

{%- if client_id==1 %}
and IS_ONE = 'Y'
{%- elif client_id==2 %}
and IS_TWO = 'Y'
{% endif %}
"""

print(Template(sql_template).render(client_id=1))
print(Template(sql_template).render(client_id=2))
print(Template(sql_template).render(client_id=3))


select * 
from claims_data
where client_id = 1
and IS_ONE = 'Y'

select * 
from claims_data
where client_id = 2
and IS_TWO = 'Y'


select * 
from claims_data
where client_id = 3


### Special loop condition variables

* `loop.last` and `loop.first` special variables
* Ex: `{% for thing in things %} {% if loop.last %} 'no comma' {% else %} 'comma' {% endif %} {% endfor %}`

In [7]:
from jinja2 import Template

# for each year, output "year_<year>"
# if it's the last year, then do not append trailing comma
sql_template = """
select
{% for year in years %}
year_{{ year }}
{%- if loop.last %}
{%- else -%}
,
{%- endif %}
{%- endfor %}

from a_table;
"""
years = [2010, 2011, 2012, 2013, 2014, 2015]

sql_code = Template(sql_template).render(years=years)
print(sql_code)


select

year_2010,
year_2011,
year_2012,
year_2013,
year_2014,
year_2015

from a_table;


### Putting it all together

Say we want to construct the following query dynamically.
* We want to parameterize the year in the integer value and the concatenated `year_<yr>` string using a variable (`{{double_curlies}}`)
* We need to loop through each year to produce the "WHEN ... THEN" lines (`{% for thing in things %}...{% endfor %}`)
* We need to loop through each year to produce the VALUES () lines after the cross-join
* Problem: the VALUES() lines need a trailing comma for all years except the last
    * So we need some kind of conditional logic to use an empty character if we're on the last iteration
    * (`{% if loop.last %}...{% endif %}`)

```sql
SELECT years.*,
       earthquakes.magnitude,
       CASE year
         WHEN 2000 THEN year_2000
         WHEN 2001 THEN year_2001
         WHEN 2002 THEN year_2002
         WHEN 2003 THEN year_2003
         WHEN 2004 THEN year_2004
         WHEN 2005 THEN year_2005
         WHEN 2006 THEN year_2006
         WHEN 2007 THEN year_2007
         WHEN 2008 THEN year_2008
         WHEN 2009 THEN year_2009
         WHEN 2010 THEN year_2010
         WHEN 2011 THEN year_2011
         WHEN 2012 THEN year_2012
         ELSE NULL END
         AS number_of_earthquakes
  FROM tutorial.worldwide_earthquakes earthquakes
 CROSS JOIN (
       SELECT year
         FROM (VALUES (2000),(2001),(2002),(2003),(2004),(2005),(2006),
                      (2007),(2008),(2009),(2010),(2011),(2012)) v(year)
       ) years
```

### Using Loops and Conditional

In [8]:
from jinja2 import Template

sql_template = """
SELECT years.*,
       earthquakes.magnitude,
       CASE year
       {%- for year  in years %}
         WHEN {{year}} THEN year_{{year}}
       {%- endfor %}
         ELSE NULL END
         AS number_of_earthquakes
  FROM tutorial.worldwide_earthquakes earthquakes
 CROSS JOIN (
       SELECT year
         FROM (VALUES {% for year in years -%}
                      ({{year}})
                      {%- if loop.last %}
                      {%- else %},{% endif %}
                      {%- endfor %}) v(year)
       ) years
"""

years = range(2000, 2020)
sql_code = Template(sql_template).render(years=years)
print(sql_code)


SELECT years.*,
       earthquakes.magnitude,
       CASE year
         WHEN 2000 THEN year_2000
         WHEN 2001 THEN year_2001
         WHEN 2002 THEN year_2002
         WHEN 2003 THEN year_2003
         WHEN 2004 THEN year_2004
         WHEN 2005 THEN year_2005
         WHEN 2006 THEN year_2006
         WHEN 2007 THEN year_2007
         WHEN 2008 THEN year_2008
         WHEN 2009 THEN year_2009
         WHEN 2010 THEN year_2010
         WHEN 2011 THEN year_2011
         WHEN 2012 THEN year_2012
         WHEN 2013 THEN year_2013
         WHEN 2014 THEN year_2014
         WHEN 2015 THEN year_2015
         WHEN 2016 THEN year_2016
         WHEN 2017 THEN year_2017
         WHEN 2018 THEN year_2018
         WHEN 2019 THEN year_2019
         ELSE NULL END
         AS number_of_earthquakes
  FROM tutorial.worldwide_earthquakes earthquakes
 CROSS JOIN (
       SELECT year
         FROM (VALUES (2000),(2001),(2002),(2003),(2004),(2005),(2006),(2007),(2008),(2009),(2010),(2011),(2012),(20

### SQL Injection

Resources
* Django Security and Preventing SQL Injection: https://django-book.readthedocs.io/en/latest/chapter20.html
* Consider using `jinjasql` library for SQL-injection-safe queries if you're constructing queries in the context of a web application: https://github.com/sripathikrishnan/jinjasql/tree/master/jinjasql
* xkcd - Bobby Tables: https://xkcd.com/327
* In short, never trust user input

In [9]:
from jinja2 import Template

sql_template = """
select *
from data_table
where user_id = {{user_id}}
"""

# expected result
input_from_user = 1
print(Template(sql_template).render(user_id=input_from_user))

# never trust user input when accepting data from a user on the internet (e.g., form data)
input_from_user = "1; DELETE FROM user_contacts WHERE 'a' = 'a'"
print(Template(sql_template).render(user_id=input_from_user))


select *
from data_table
where user_id = 1

select *
from data_table
where user_id = 1; DELETE FROM user_contacts WHERE 'a' = 'a'
